In [ ]:
# import the necessary packages
from imutils.video import VideoStream
import numpy as np
import sys
from PIL import Image
import argparse
import imutils
import time
import cv2
from skimage import img_as_ubyte
import os,glob

num = 1
def background_sub(c_dup,crop):
    def trim(frame):
        global count
        #crop top
        if not np.sum(frame[0]):
            count+=1
            return trim(frame[1:])
        return frame

    global count
    count=0
    gray = cv2.cvtColor(c_dup, cv2.COLOR_BGR2GRAY) 
    ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV +cv2.THRESH_OTSU)
    image = trim(thresh)
    if(count>10):
        img_resized = crop[count-10:]
    elif(count>5 and count<=10):
        img_resized = crop[count-4:]
    else:
        img_resized = crop[:]
    return img_resized


# initialize the list of class labels MobileNet SSD was trained to
# detect, then generate a set of bounding box colors for each class
CLASSES = ["background", "aeroplane", "bicycle", "bird", "boat","bottle", "bus", "car", "cat", "chair", "cow", "diningtable","dog", "horse", "motorbike", "person", "pottedplant", "sheep","sofa", "train", "tvmonitor"]

IGNORE = set(["motorbike"])

COLORS = np.random.uniform(0, 255, size=(len(CLASSES), 3))

model = "MobileNetSSD_deploy.caffemodel"
prototxt = "MobileNetSSD_deploy.prototxt.txt"
conf = 0.2
# load our serialized model from disk
print("[INFO] loading model...")
net = cv2.dnn.readNetFromCaffe(prototxt, model)

# initialize the video stream, allow the cammera sensor to warmup,
print("[INFO] starting video stream...")

vs = cv2.VideoCapture('videos/1.mp4')

#time.sleep(2.0)

detected_objects = []
kill=0
# loop over the frames from the video stream
loop = 0
while True:
    # grab the frame from the threaded video stream and resize it
    # to have a maximum width of 400 pixels
    ret, fr = vs.read()
    if(fr is not None):
        frame = np.array(fr, dtype=np.uint8)
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = imutils.resize(frame, width=800)
        # grab the frame dimensions and convert it to a blob
        (h, w) = frame.shape[:2]
        blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)),0.007843, (300, 300), 127.5)

        # pass the blob through the network and obtain the detections and
        # predictions
        net.setInput(blob)
        detections = net.forward()

        # loop over the detections
        if(loop %12 != 0 ):
            pass
        else:
            for i in np.arange(0, detections.shape[2]):
                #print(i)
                #break
                # extract the confidence (i.e., probability) associated with
                # the prediction
                confidence = detections[0, 0, i, 2]

                # filter out weak detections by ensuring the `confidence` is
                # greater than the minimum confidence
                if confidence > conf:
                    # extract the index of the class label from the
                    # `detections`, then compute the (x, y)-coordinates of
                    # the bounding box for the object
                    idx = int(detections[0, 0, i, 1])
                    if((CLASSES[idx] not in IGNORE) or (confidence<0.30)):
                        continue
                    else:

                        box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                        (startX, startY, endX, endY) = box.astype("int")
                        #bike_dims = frame[startY:endY,startX:endX]
                        cropped = frame[startY-75:endY-5,startX+30:endX-30]
                        cropped_dup = frame[startY-75:endY-5,startX+60:endX-60]

                        #cropped = frame[startY-15:endY,startX:endX]
                        p1 = 'video_to_images/my_'+str(num)+'.png'
                        p2 = 'video_to_images/my_'+str(num)+'_'+str(num)+'.png'
                        #p3 = 'my_'+str(num)+'_'+'bike'+'.png'
                        if(len(cropped_dup)!=0):
                            try:
                                #print("lengths: ",len(cropped),len(cropped_dup))
                                img1 = Image.fromarray(cropped_dup, 'RGB')
                                img2 = Image.fromarray(cropped, 'RGB')
                                #img3 = Image.fromarray(bike_dims, 'RGB')
                                img1.save(p1)
                                img2.save(p2)
                                #img3.save(p3)
                                im_p1 = cv2.imread(p1)
                                im_p2 = cv2.imread(p2)

                                #cv2.waitKey(2000)
                                num+=1
                            except:
                                print("error")
                            img_resized = background_sub(im_p1,im_p2)
                            #print(img_resized)
                            cv2.imwrite(p1,img_resized)
                        # draw the prediction on the frame
                        label = "{}: {:.2f}%".format(CLASSES[idx],confidence * 100)
                        detected_objects.append(label)
                        cv2.rectangle(frame, (startX, startY), (endX, endY),COLORS[idx], 2)
                        y = startY - 15 if startY - 15 > 15 else startY + 15
                        cv2.putText(frame, label, (startX, y),cv2.FONT_HERSHEY_SIMPLEX, 0.5, COLORS[idx], 2)
                        #break
            #break
        loop+=1
        # show the output frame
        cv2.imshow("Frame", frame)
        key = cv2.waitKey(1) & 0xFF
        # if the `q` key was pressed, break from the loop
        if key == ord("q"):
            break
'''
files = glob.glob("*_*_*[!bike].png")
print(files)
for file in files:
	os.remove(file)
'''

[INFO] loading model...
[INFO] starting video stream...
error
error


In [ ]:
import cv2
import os
import numpy as np
from skimage.io import imread_collection

images = imread_collection('video_to_images/*.png')
print(images)
skip=0
ok = []
not_ok = []
sub_ele = 0
user_num = 0
first_num = 0
first_img = images[0]
newpath = 'Cluster_Dataset/user_'+str(user_num)
if not os.path.exists(newpath):
    os.makedirs(newpath)

cv2.imwrite(newpath+"/"+str(sub_ele)+'.png',cv2.cvtColor(first_img, cv2.COLOR_RGB2BGR))
sub_ele+=1
num=0
ok.append(num)
correct=0
print("*****************")
print("update number(first): ",num)
while(True):
    num += 1
    print("update number(second): ",num)
    if(num==len(images)):
        print("*******************\n")
        print("\n*******************")
        print("Done with 'Grouping_Images'")
        print("*******************")
        break
    second_img = images[num]
    # 1) Check if 2 images are equals
    if first_img.shape == second_img.shape:
        print("The images have same size and channels")
        difference = cv2.subtract(first_img, second_img)
        b, g, r = cv2.split(difference)
        if cv2.countNonZero(b) == 0 and cv2.countNonZero(g) == 0 and cv2.countNonZero(r) == 0:
            print("The images are completely Equal")
        else:
            print("The images are NOT equal")
    # 2) Check for similarities between the 2 images
    sift = cv2.xfeatures2d.SIFT_create()
    kp_1, desc_1 = sift.detectAndCompute(first_img, None)
    kp_2, desc_2 = sift.detectAndCompute(second_img, None)
    index_params = dict(algorithm=0, trees=5)
    search_params = dict()
    flann = cv2.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(desc_1, desc_2, k=2)
    good_points = []
    ratio = 0.6
    for m, n in matches:
        if m.distance < ratio*n.distance:
            good_points.append(m)
    print("total good_points: ",len(good_points))
    if(len(good_points)>=10):
        print("correct number: ",num)
        cv2.imwrite(newpath+"/"+str(sub_ele)+'.png',cv2.cvtColor(second_img, cv2.COLOR_RGB2BGR))
        sub_ele+=1
        ok.append(num)
        correct=num
        skip+=1
        wrong_preds = 0
    else:
        wrong_preds += 1 
        skip+=1
        if(num not in ok):
            not_ok.append(num)
        if(wrong_preds>1):
            print("wrong preds: ",wrong_preds)
            print("total skips: ",skip)
            print("current number: ",num)
            print("user cluster is completed")
            print("ok list: ",ok)
            print("not_ok: ",not_ok)
            print("*******************")
            sub_ele = 0
            user_num+=1
            first_img = images[not_ok[0]]
            num = not_ok[0]
            print("update number(first): {}".format(num))
            not_ok = []
            newpath = 'Cluster_Dataset/user_'+str(user_num)
            if not os.path.exists(newpath):
                os.makedirs(newpath)
            cv2.imwrite(newpath+"/"+str(sub_ele)+'.png',cv2.cvtColor(first_img, cv2.COLOR_RGB2BGR))
            ok.append(num)
            sub_ele+=1
            skip=0
            wrong_preds = 0
        else:
            print("user cluster is 'not at completed'")
    #result = cv2.drawMatches(first_img, kp_1, second_img, kp_2, good_points, None)

    #cv2.imshow("result", result)
    #cv2.imshow("first_img", first_img)
    #cv2.imshow("Duplicate", second_img)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()

In [1]:
import cv2,os
import glob

files = glob.glob("Cluster_Dataset/*")
num=0
for file in files:
	#num=0
	c_file = file+'/cropped/'
	if not os.path.exists(c_file):
		os.makedirs(c_file)
	images = glob.glob(file+'/*.png')
	for image in images:
		num+=1
		img = cv2.imread(image)
		height,width,channels = img.shape
		h = int((height*25)/100)
		x,y = 10,1
		crop_img = img[y:h, x:width]
		#print(c_file+str(num)+'.png')
		cv2.imwrite(c_file+str(num)+'.png',crop_img)
		#cv2.imshow("cropped", crop_img)
		#cv2.waitKey(0)

In [2]:
import os
import glob
import cv2
from keras.preprocessing import image
from keras.models import load_model
print(os.getcwd())
from PIL import Image
import numpy as np
import warnings
warnings.simplefilter('ignore')

C:\Users\adity\Desktop\Final Year Project


In [3]:
model = load_model('helmet_detection_model.hdf5')#new_4-24-1(0.10 0.96 0.23 0.93).h5

In [4]:
l1 = [1,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,1,1,1,1,0,1,1,1,0,0,0,1,1,1,1,1,1,1,1,1,0,1,1,1,1]

In [5]:
len(l1)

42

In [6]:
import glob
from PIL import Image
pis=[]
files = glob.glob("Cluster_Dataset\\*.png")
print(files,len(files))
#assert(False)
for file in files:
    im = image.load_img(file)
    width,height = im.size
    f, e = os.path.splitext(file)
    im = im.crop((1, 1, width, height//4))
    imResize = im.resize((160,160), Image.ANTIALIAS)
    imResize = imResize.convert('RGB')
    #imResize.save(f+'_1.png', 'PNG', quality=100)
    img = image.img_to_array(imResize)
    im_f = np.expand_dims(img,axis=0)
    result = model.predict(im_f)
    #print(result)
    #print(int(result[0][0]))
    pis.append(int(result[0][0]))
    #print(f.split('\\')[-1],result[0])
pis

[] 0


[]

In [7]:
np.array(l1).ravel()

array([1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1])

In [8]:
np.array(pis).ravel()

array([], dtype=float64)

In [9]:
for i in range(len(files)):
    print(files[i].split('\\')[1]+ '==> ' + str(pis[i])+'==> '+str(l1[i]))

In [10]:
import glob
from PIL import Image
from keras.preprocessing import image
import os
user_preds = []
h1 = 0
h2 = 0
user_folders = glob.glob("Cluster_Dataset\\*[!.png]")
print(user_folders)
for user in user_folders:
    uname = user.split('\\')[-1]
    print(uname,end=' => ')
    cropped_files = glob.glob(user+'\\cropped\\*')
    print('[',end='')
    res = 'no_helmet'
    for file in cropped_files:
        im = image.load_img(file)
        width,height = im.size
        f, e = os.path.splitext(file)
        im = im.crop((1, 1, width, height//4))
        imResize = im.resize((82,82), Image.ANTIALIAS)
        imResize = imResize.convert('RGB')
        #imResize.save(f+'_1.png', 'PNG', quality=100)
        img = image.img_to_array(imResize)
        im_f = np.expand_dims(img,axis=0)
        result = model.predict(im_f)
        if(result[0][0]>=0.6):
            res='helmet'
        print(result[0][0],end=' ')
    print(']',end=' => ')
    print(res,end='\n')
    if(res == 'helmet'):
        print
        pis.append(1)
    elif(res == 'no_helmet'):
        pis.append(0)
pis

[]


[]